# Imports

In [142]:
import cv2
import numpy as np
import os

# Função para normalizar a imagem para 28x28

In [143]:
def ImgNormalizer(img):
    w = len(img[0])
    h = len(img[1])
    Draw = []
    for x in range(w):
        for y in range(h):
            if img[x][y] == 255:
                print(x,y)
                Draw.append([x,y])

    # Inicia os valores mínimos e máximos com os primeiros valores
    MaxXValue = Draw[0][0]
    MinXValue = Draw[0][0]
    MaxYValue = Draw[0][1]
    MinYValue = Draw[0][1]
    for pair in Draw[1:]:
        if pair[0] > MaxXValue:
            MaxXValue = pair[0]
        if pair[0] <= MinXValue:
            MinXValue = pair[0]
        if pair[1] > MaxYValue:
            MaxYValue = pair[1]
        if pair[1] <= MinYValue:
            MinYValue = pair[1]

    # Calcula as amplitudes
    XAmplitude = MaxXValue - MinXValue
    YAmplitude = MaxYValue - MinYValue
    
    if XAmplitude > YAmplitude:
        # Altera as coordenadas para ir de 0 ate a aplitude maxima e centraliza a amplitude menor
        Amp = XAmplitude
        for pair2 in Draw:
            pair2[0] = pair2[0] - MinXValue
            pair2[1] = pair2[1] - MinYValue + (Amp/2) - (YAmplitude/2)
    else:
        # Altera as coordenadas para ir de 0 ate a aplitude maxima e centraliza a amplitude menor
        Amp = YAmplitude
        for pair2 in Draw:
            pair2[0] = pair2[0] - MinXValue + (Amp/2) - (XAmplitude/2)
            pair2[1] = pair2[1] - MinYValue
    
    Draw2 = []
    for pair3 in Draw:
        # Reduz a amplitude para 28x28 com 2 pixels de borda e arredonda os resultados
        newpair = []
        newpair.append(((23/Amp)*pair3[0]) + 2)
        newpair.append(((23/Amp)*pair3[1]) + 2) 
        newpairR = np.int32(np.rint(newpair))
        Draw2.append(newpairR)
    print("Coordenadas apos normalização e arredondamento")
    print(Draw2)
    # Deixa apenas os pares únicos, excluindo os repetidos após o arredondamento
    Draw2 = np.unique(Draw2,axis=0)
    print(Draw2)

    #Desenha a imagem 28x28
    imgSmall = np.full((28,28), 0, dtype=np.uint8)
    imgSmall2 = np.full((28,28), 0, dtype=np.uint8)
    for pair in Draw2:
        imgSmall[pair[0]][pair[1]] = 255
        cv2.circle(imgSmall2,(pair[1],pair[0]),1,(255,255,255),-1)

    return imgSmall,imgSmall2

# Classe geradora do desenho

In [144]:
class DrawingApp:
    def __init__(self,root_path):
        # Variáveis iniciais
        self.root_path = root_path
        self.isDrawing = False
        self.count = int(4)
    
    def drawLine(self,event,x,y,flags,param):
        #Desenha na imagem enquanto o mouse se move após o botão esquerdo do mouse é pressionado
        #Desenha em 2 imagens ao mesmo tempo: Uma para ter uma referencia do desenho e uma para gerar as coordenadas do mouse
        img1 = param[0]
        img2 = param[1]
        if event == cv2.EVENT_LBUTTONDOWN:
            self.isDrawing = True
        elif event == cv2.EVENT_MOUSEMOVE and self.isDrawing:
            cv2.circle(img1,(x,y),5,(255,255,255),-1)
            cv2.circle(img2,(x,y),0,(255,255,255),-1)
        elif event == cv2.EVENT_LBUTTONUP:
            self.isDrawing = False

    def run(self):
        # Imagem base vazia
        img = np.full((640,640), 0, dtype=np.uint8)
        img2 = np.full((640,640), 0, dtype=np.uint8)

        images = [img,img2]

        #gera o nome da janela
        windowName = 'drawing app'
        cv2.namedWindow(windowName)

        #chama a função de desenho que chama a função drawLine com os eventos do mouse
        cv2.setMouseCallback(windowName,self.drawLine,images)

        while True:
            cv2.imshow(windowName,images[0])
            wait = cv2.waitKey(1)
            if wait == ord('q'):
                print("saiu")
                break
            elif wait == ord('r'):
                print("entrou no reset")
                images[0] = np.full((640,640), 0, dtype=np.uint8)
                images[1] = np.full((640,640), 0, dtype=np.uint8)
            elif wait == ord('s'):
                print("entrou no save")
                imgSaveName = str(self.count)+'.jpg'
                SavePath = os.path.join(self.root_path,'Base1\\'+imgSaveName)
                if not cv2.imwrite(SavePath, images[0]):
                    raise Exception("Could not write the image")
                
                imgSaveNamedots = str(self.count)+'dots'+'.jpg'
                SavePath = os.path.join(self.root_path,'Base1\\'+imgSaveNamedots)
                if not cv2.imwrite(SavePath, images[1]):
                    raise Exception("Could not write the image")
                
                imagesSmall = ImgNormalizer(images[1])
                imgSave28thin = str(self.count)+'SmallThin'+'.jpg'
                SavePath = os.path.join(self.root_path,'Base1\\'+imgSave28thin)
                if not cv2.imwrite(SavePath, imagesSmall[0]):
                    raise Exception("Could not write the image")
                imgSave28bold = str(self.count)+'SmallBold'+'.jpg'
                SavePath = os.path.join(self.root_path,'Base1\\'+imgSave28bold)
                if not cv2.imwrite(SavePath, imagesSmall[1]):
                    raise Exception("Could not write the image")
                
                self.count = self.count + 1

        cv2.destroyAllWindows()

# Função Chamada para desenhar

In [145]:
def Drawing():
    root = os.getcwd()
    app = DrawingApp(root)
    app.run()

# Execução

In [146]:
if __name__ == '__main__':
    Drawing()

saiu


In [147]:
a = []
a.append(2)
a.append(3)
a

[2, 3]